# 02. Neuralnet Q Learning example

Neuralnet Q Learning을 실습해봅니다.
- 신경망의 parameter(weight)를 업데이트 함에 따라 state에 대한 Q value가 변화합니다.
- Q Learning의 TD error를 loss function으로 하여 학습합니다.

## Collab 용 package 설치 코드

In [ ]:
!pip install gym

In [ ]:
import tensorflow as tf
import numpy as np
import random
import gym

import matplotlib.pyplot as plt
%matplotlib inline

np.random.seed(285)
tf.set_random_seed(285)

print("tensorflow version: ", tf.__version__)
print("gym version: ", gym.__version__)

## Frozen Lake

**[state]**

        SFFF
        FHFH
        FFFH
        HFFG

    S : starting point, safe
    F : frozen surface, safe
    H : hole, fall to your doom
    G : goal, where the frisbee is located
    
**[action]**

    LEFT = 0
    DOWN = 1
    RIGHT = 2
    UP = 3

### Frozen Lake (not Slippery)

In [ ]:
def register_frozen_lake_not_slippery(name):
    from gym.envs.registration import register
    register(
        id=name,
        entry_point='gym.envs.toy_text:FrozenLakeEnv',
        kwargs={'map_name' : '4x4', 'is_slippery': False},
        max_episode_steps=100,
        reward_threshold=0.78, # optimum = .8196
    )

register_frozen_lake_not_slippery('FrozenLakeNotSlippery-v0')

In [ ]:
# Load Environment
env = gym.make('FrozenLakeNotSlippery-v0')
state_size = env.observation_space.n
action_size = env.action_space.n
print("State_size : ", state_size)
print("Action_size: ",action_size)

## Q-Learning using Neural Network  
**Update 식**  
  
$J(w) = \mathbb{E}_{\pi}[(target - \hat q(S,A,w))^2]$  
  
$ \begin{align} \Delta w & = - \frac{1}{2} \alpha \nabla_w J(w)
\\ & = \alpha (R_{t+1} + \gamma max(\hat q(S_{t+1},a ,w)) - \hat q(S_{t},A_{t},w))\nabla_w \hat q(S_{t},A_{t},w) \end{align}$

### 학습 순서
1. 초기 state 받음 (env.reset())
2. action 선택 (e - greedy policy)
3. 선택한 action으로 다음 state로 이동 (env.step())
4. 다음 state와 reward를 이용해 update식 작성
5. 신경망 업데이트
6. 반복

## Tensorflow 코드 흐름
1. 각 연산자에 대한 그래프를 구성한다.
2. 실제 데이터를 그래프에 할당하면서 전체 그래프를 실행한다.

### Build graph

In [ ]:
# placeholder 선언
# state
inputs = tf.placeholder(shape=[1], dtype=tf.int64)
# state에 대한 action
input_action = tf.placeholder(shape=[1], dtype=tf.int64)
# Loss 식의 target
target = tf.placeholder(shape=[1], dtype=tf.float32)

layers = tf.contrib.layers

# 신경망 구성 함수
# one-hot vector : 입력 1을 단순한 숫자로 받는 것보다 [1, 0, 0, 0] 처럼 encoding된 값으로 바꾸어 받는 것이
#                       학습에 유리하다. 모든 입력이 크기에 관계없이 동등해지게 된다.
# tf.one_hot( 입력, one-hot size )
def build_network(inputs):   
    with tf.variable_scope('q_net'):
        # 빈칸 {} 을 지우고 채워주세요.
        input_onehot = tf.one_hot({}, {}, dtype=tf.float32)
        fc1 = layers.fully_connected(inputs={},
                                                  num_outputs={},
                                                  activation_fn=None)
    return fc1

# 신경망 구성
q_value = build_network(inputs)

# 현재 action에 대한 Q_value 구하는 연산
# q_value = [1, 2, 3, 4] curr_action = [0, 1, 0, 0] --(원소 곱)--> [0, 2, 0, 0] --(sum)--> [2]
curr_action = tf.one_hot(input_action, action_size)
curr_q_value = tf.reduce_sum(tf.multiply(q_value, curr_action))

# Loss 함수 구성
# 직접 구현해보세요. ( learning_rate = 0.1 )
# 참고) 제곱 : tf.square()     
#          optimizer : tf.train.GradientDescentOptimizer( learning_rate )
loss_op = 
opt = 
train_op = opt.minimize(loss_op)

### Executing a graph in a tf.Session

In [ ]:
# Session 열기
sess = tf.Session()

# 변수 초기화
sess.run(tf.global_variables_initializer())

# 변수(파라미터) 확인
for var in tf.trainable_variables('q_net'):
    print(var)
    print(sess.run(var))

### Action select using Q value

In [ ]:
# 초기 state
state = env.reset()
state = np.reshape(state, [1])
print("현재 state:", state)

# 현재 state에 대한 Q-value
# 빈칸 {}을 채워보세요.
# 참고) sess.run( "Q-value를 구하는 신경망 그래프", feed_dict={inputs: "신경망 입력"} )
curr_q = sess.run({}, feed_dict={inputs: {} })
print("현재 state의 Q value:", curr_q)

In [ ]:
# action 선택 ( greedy )
# 직접 구현해보세요.
action = 
print("선택한 action: ", action)

### 선택한 Action으로 다음 State, Reward 받기

In [ ]:
# action을 이용해 env.step하기
# 빈칸 {} 을 채워보세요.
next_state, reward, done, _ = env.step({})
next_state = np.reshape(next_state, [1])
print(next_state)

### update를 위한 (미래)보상 값(target) 계산

In [ ]:
gamma = 0.9
if done:
    next_q_value = np.array([reward])
else:
    # 직접 작성해보세요.
    # 위 수식 참고.
    # 참고) R + gamma * next state의 q-value 중 max
    next_q_value = 
print(next_q_value)

### Update Neural Net

In [ ]:
action = np.reshape(action, [1])

# train_op를 sess.run 하여 학습 실행.
# 빈칸 {} 을 채워보세요.
loss, _ = sess.run([loss_op, train_op], feed_dict={inputs: {}, target: {}, input_action: {}})
print(loss)

for var in tf.trainable_variables('q_net'):
    print(var)
    print(sess.run(var))

### 학습 시작

In [ ]:
rlist = []
slist = []
epsilon_list = []

EPISODE = 2000
gamma = 0.99

update_count = 0
loss_list = []

# Episode 수만큼 반복
for step in range(EPISODE):
    epsilon = 1. / ((step/50)+10)
    epsilon_list.append(epsilon)
    
    # 초기 state
    state = env.reset()
    state = np.reshape(state, [1])
    print("[Episode {}]".format(step))
    total_reward = 0
    limit = 0
    done = False
    
    while not done and limit < 99:
        # 위에서 했던 코드를 참조하여 아래 학습 코드를 작성해보세요.
        # 현재 state의 Q value불러오기
        curr_q = 
        
        # e-greedy policy로 action 선택
        if epsilon > np.random.random():
            # random
            action = env.action_space.sample()
        else:
            # greedy
            action =  
            
        # 선택한 action으로 env.step 하기
        next_state, reward, done, _ = 
        next_state = np.reshape(next_state, [1])
                          
        if reward == 1.0:
            print("GOAL")
        
        # 업데이트를 위한 (미래)보상값 반환
        # episode가 끝났다면
        if done:
            next_q_value = 
        # 끝나지 않았다면
        else:
            next_q_value = 
        
        # Q update
        action = np.reshape(action, [1])
        loss, _ = 
        
        loss_list.append(loss)
        update_count += 1
        
        slist.append(state.item())
        state = next_state
        total_reward += reward
        limit += 1
        
    print(slist)
    slist = []
    print("total reward: ", total_reward)
    rlist.append(total_reward)
    
print("성공한 확률" + str(sum(rlist) / EPISODE) + "%")

In [ ]:
for var in tf.trainable_variables('q_net'):
    print(var)
    print(sess.run(var))

In [ ]:
# epsilon 변화 그래프
steps = np.arange(EPISODE)
plt.title('Epsilon values')
plt.xlabel('Timestep')
plt.ylabel('$\\epsilon$')
plt.plot(steps, epsilon_list)

In [ ]:
# loss 변화 그래프
update_count = np.arange(update_count)
plt.title('Loss values')
plt.xlabel('Update Count')
plt.ylabel('Loss')
plt.plot(update_count, loss_list)

### Test agent

In [ ]:
state = env.reset()
state = np.reshape(state, [1])
done = False
limit = 0

epsilon = 0.0
while not done and limit < 30:
    # 학습된 신경망을 테스트하는 코드를 작성해보세요.
    curr_q = 
    action = 
    next_state, reward, done, _ =
    next_state = np.reshape(next_state, [1])
    
    env.render()
    state = next_state
    limit += 1